In [23]:
# Main imports
from econml.metalearners import TLearner, SLearner, XLearner, DomainAdaptationLearner

# Helper imports 
import numpy as np
import pandas as pd
from numpy.random import binomial, multivariate_normal, normal, uniform
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingRegressor
import matplotlib.pyplot as plt
import plotly.graph_objects as go

%matplotlib inline

In [2]:
# Define DGP
def generate_data(n, d, controls_outcome, treatment_effect, propensity):
    """Generates population data for given untreated_outcome, treatment_effect and propensity functions.
    
    Parameters
    ----------
        n (int): population size
        d (int): number of covariates
        controls_outcome (func): untreated outcome conditional on covariates
        treatment_effect (func): treatment effect conditional on covariates
        propensity (func): probability of treatment conditional on covariates
    """
    # Generate covariates
    X = multivariate_normal(np.zeros(d), np.diag(np.ones(d)), n)
    # Generate treatment
    T = np.apply_along_axis(lambda x: binomial(1, propensity(x), 1)[0], 1, X)
    # Calculate outcome
    Y0 = np.apply_along_axis(lambda x: controls_outcome(x), 1, X)
    treat_effect = np.apply_along_axis(lambda x: treatment_effect(x), 1, X)
    Y = Y0 + treat_effect * T
    return (Y, T, X)

In [3]:
# controls outcome, treatment effect, propensity definitions
def generate_controls_outcome(d):
    beta = uniform(-3, 3, d)
    return lambda x: np.dot(x, beta) + normal(0, 1)
treatment_effect = lambda x: (1 if x[1] > 0.1 else 0)*8
propensity = lambda x: (0.8 if (x[2]>-0.5 and x[2]<0.5) else 0.2)

In [4]:
# DGP constants and test data
d = 5
n = 1000
n_test = 250
controls_outcome = generate_controls_outcome(d)
X_test = multivariate_normal(np.zeros(d), np.diag(np.ones(d)), n_test)
delta = 6/n_test
X_test[:, 1] = np.arange(-3, 3, delta)

In [5]:
Y, T, X = generate_data(n, d, controls_outcome, treatment_effect, propensity)

In [6]:
X_df = pd.DataFrame(X, columns = ["X_1", "X_2", "X_3", "X_4", "X_5"])
X_df.head()

,X_1,X_2,X_3,X_4,X_5
0,-2.360562,-0.541052,-1.674669,-0.301408,0.055745
1,2.591409,-0.600360,-0.653905,1.270261,0.508157
2,-0.675472,-0.816530,0.570876,0.914654,-1.346066
3,-0.049682,0.063574,0.940947,-0.985525,0.817075
4,0.476385,-0.788016,0.045483,0.500688,-0.864267


In [7]:
data_dict = dict({"Y":Y, "T": T})
df = pd.DataFrame(data_dict)
df = pd.concat([df, X_df], axis=1)
df.head(5)

,Y,T,X_1,X_2,X_3,X_4,X_5
0,1.567376,0,-2.360562,-0.541052,-1.674669,-0.301408,0.055745
1,3.915212,0,2.591409,-0.600360,-0.653905,1.270261,0.508157
2,-1.715072,0,-0.675472,-0.816530,0.570876,0.914654,-1.346066
3,-0.654889,1,-0.049682,0.063574,0.940947,-0.985525,0.817075
4,-1.476838,0,0.476385,-0.788016,0.045483,0.500688,-0.864267


# T-Learner

In [16]:
# 集団を2つに分ける
df_0 = df.query("T == 0")  # 介入を受けていない集団
df_1 = df.query("T == 1")  # 介入を受けた集団

In [17]:
# ランダムフォレストモデルを作成
from sklearn.ensemble import RandomForestRegressor

# 介入を受けていないモデル
reg_0 =  GradientBoostingRegressor(n_estimators=100, max_depth=6, min_samples_leaf=int(n/100))
reg_0.fit(df_0[["X_1", "X_2", "X_3", "X_4", "X_5"]], df_0[["Y"]])

# 介入を受けたモデル
reg_1 = models = GradientBoostingRegressor(n_estimators=100, max_depth=6, min_samples_leaf=int(n/100))
reg_1.fit(df_1[["X_1", "X_2", "X_3", "X_4", "X_5"]], df_1[["Y"]])

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


GradientBoostingRegressor(max_depth=6, min_samples_leaf=10)

In [33]:
# ATEを求める
mu_0 = reg_0.predict(X_test)
mu_1 = reg_1.predict(X_test)

ATE = (mu_1-mu_0).mean()
print("ATE：", ATE)

ATE： 3.7757906834502504


In [34]:
# 推定された治療効果を各人ごとに求めます
t_estimated = reg_1.predict(X_test) - reg_0.predict(X_test)

In [35]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x = X_test[:, 1],
        y = t_estimated,
        mode = "markers"
    )
)
